<a href="https://colab.research.google.com/github/AadishSethiya/DiffieHellman/blob/main/task_2/210100001_task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re

## Reading from the excel file and observing the dataset

In [2]:
df = pd.read_excel('corpo_announcements.xlsx')
print("Size of the dataset is", df.shape)
df.head()

Size of the dataset is (29268, 28)


,ATTACHMENTNAME,HEADLINE,NEWSID,NEWSSUB,_company,AGENDA_ID,ANNOUNCEMENT_TYPE,CATEGORYNAME,CRITICALNEWS,DT_TM,...,RN,SCRIP_CD,SLONGNAME,TimeDiff,TotalPageCnt,XML_NAME,disseminated_time,last_update,received_time,SOURCE
0,9982e28d-ec20-4a40-a743-81f2d6d2eb86.pdf,Issue of Duplicate Share Certificates(s),d912cfc6-8a9c-4dbc-8d47-d0ea55d7bc56,PUNJAB NATIONAL BANK - 532461 - Compliances-Re...,pnb,177,A,NaN,0,2022-09-26T17:36:03.993,...,1,532461,PUNJAB NATIONAL BANK,00:00:00,17,ANN_532461_D912CFC6-8A9C-4DBC-8D47-D0EA55D7BC56,2022-09-26 17:36:03,2022-09-26 18:02:06,2022-09-26 17:36:03,https://www.bseindia.com/xml-data/corpfiling/A...
1,f7b9cd02-7cac-4a0d-9989-0e8457929015.pdf,This is to inform you that pursuant to Regulat...,7cf81fb3-b73e-4806-aae7-8ed20d548b9f,BALKRISHNA INDUSTRIES LTD.-$ - 502355 - Compli...,balkrisind,228,A,NaN,0,2022-09-26T17:33:17.907,...,1,502355,BALKRISHNA INDUSTRIES LTD.-$,00:00:00,17,ANN_502355_7CF81FB3-B73E-4806-AAE7-8ED20D548B9F,2022-09-26 17:33:17,2022-09-26 18:02:06,2022-09-26 17:33:17,https://www.bseindia.com/xml-data/corpfiling/A...
2,6521d50d-c0f4-495d-8c9a-02e72c2dba63.pdf,Indian Energy Exchange Limited has informed th...,a6801b26-873c-4df5-8423-f64796fb2f6f,Indian Energy Exchange Ltd - 540750 - Closure ...,iex,66,A,Insider Trading / SAST,0,2022-09-26T17:33:04.163,...,1,540750,Indian Energy Exchange Ltd,00:00:00,17,ANN_540750_A6801B26-873C-4DF5-8423-F64796FB2F6F,2022-09-26 17:33:04,2022-09-26 18:02:06,2022-09-26 17:33:04,https://www.bseindia.com/xml-data/corpfiling/A...
3,8f71f319-94c8-4c97-965b-e806299e72a7.pdf,Intimation of loss of share certificate and st...,92198915-9f2e-4afb-b5fd-0c9d1c9ce8a2,ELGI EQUIPMENTS LTD. - 522074 - Compliances-Re...,elgiequip,177,A,NaN,0,2022-09-26T17:28:10.77,...,1,522074,ELGI EQUIPMENTS LTD.,00:00:00,17,ANN_522074_92198915-9F2E-4AFB-B5FD-0C9D1C9CE8A2,2022-09-26 17:28:10,2022-09-26 18:02:06,2022-09-26 17:28:10,https://www.bseindia.com/xml-data/corpfiling/A...
4,ea1f74b1-3cb7-4a09-a5cd-7749354013d4.pdf,Pursuant to Reg 30 read with Schedule III of S...,f3fcfc77-d5de-4450-9124-ac037f4dc7e1,Bharat Dynamics Ltd - 541143 - Shareholder Mee...,bdl,3,A,AGM/EGM,1,2022-09-26T17:27:20.457,...,1,541143,Bharat Dynamics Ltd,00:00:00,17,ANN_541143_F3FCFC77-D5DE-4450-9124-AC037F4DC7E1,2022-09-26 17:27:20,2022-09-26 18:02:06,2022-09-26 17:27:20,https://www.bseindia.com/xml-data/corpfiling/A...


### Observing all the columns the dataframe has

In [3]:
df.columns

Index(['ATTACHMENTNAME', 'HEADLINE', 'NEWSID', 'NEWSSUB', '_company',
       'AGENDA_ID', 'ANNOUNCEMENT_TYPE', 'CATEGORYNAME', 'CRITICALNEWS',
       'DT_TM', 'FILESTATUS', 'Fld_Attachsize', 'MORE', 'NEWS_DT', 'NSURL',
       'OLD', 'PDFFLAG', 'QUARTER_ID', 'RN', 'SCRIP_CD', 'SLONGNAME',
       'TimeDiff', 'TotalPageCnt', 'XML_NAME', 'disseminated_time',
       'last_update', 'received_time', 'SOURCE'],
      dtype='object')

## Getting earnings call transcripts
Upon deep analysis, we can observe that all the datapoints that have an earnings call transcript has the following notice in the <span style="color:yellow">NEWSSUB</span> column, has the text <span style="color:red">Announcement under Regulation 30 (LODR)-Earnings Call Transcript</span>. So we just have to look at all the data points with this feature and then the given pdf url in <span style="color:yellow">SOURCE</span> column is the transcript for that earnings call. This helps us get 531 data points. This transcript links are specified in the <span style="color:yellow">TRANSCRIPTS</span> column.

In [4]:
pd.set_option('display.max_colwidth', None)
transcript_df = df[df['NEWSSUB'].str.contains('transcript',case=False)]
transcript_df = transcript_df[transcript_df['NEWSSUB'].str.contains('call',case=False)]
transcript_df = transcript_df.reset_index(drop=True)
transcript_df = transcript_df.rename(columns={'SOURCE':"TRANSCRIPTS"})
transcript_df['AUDIO'] = np.nan
audio_df = transcript_df[transcript_df['HEADLINE'].str.contains('audio',case=False)]
transcript_df

,ATTACHMENTNAME,HEADLINE,NEWSID,NEWSSUB,_company,AGENDA_ID,ANNOUNCEMENT_TYPE,CATEGORYNAME,CRITICALNEWS,DT_TM,...,SCRIP_CD,SLONGNAME,TimeDiff,TotalPageCnt,XML_NAME,disseminated_time,last_update,received_time,TRANSCRIPTS,AUDIO
0,45a5514f-9e10-4004-b787-a093bfac4307.pdf,Transcript of Conference Call held on 12.10.2022,23936ba4-1ab4-4b38-943a-936fe8e2b8db,DELTA CORP LTD. - 532848 - Announcement under Regulation 30 (LODR)-Earnings Call Transcript,deltacorp,157,A,Company Update,0,2022-10-14T16:55:19.01,...,532848,DELTA CORP LTD.,00:00:01,24,ANN_532848_23936BA4-1AB4-4B38-943A-936FE8E2B8DB,2022-10-14 16:55:19,2022-10-14 18:21:23,2022-10-14 16:55:18,https://www.bseindia.com/xml-data/corpfiling/AttachLive/45a5514f-9e10-4004-b787-a093bfac4307.pdf,NaN
1,745d6bd9-e798-419a-8d49-b6beef99ab47.pdf,"Pursuant to Regulation 30 of the SEBI (Listing Obligations and Disclosure Requirements) Regulations, 2015, please find enclosed the transcript of the earnings conference call for the quarter ended September 30, 2022 conducted after the meeting of Board of Directors held on October 10, 2022, for your information and records.",fec1e41a-89a8-4220-91a6-bbde589af98e,TATA CONSULTANCY SERVICES LTD. - 532540 - Announcement under Regulation 30 (LODR)-Earnings Call Transcript,tcs,157,A,Company Update,0,2022-10-14T18:31:34.29,...,532540,TATA CONSULTANCY SERVICES LTD.,00:00:00,24,ANN_532540_FEC1E41A-89A8-4220-91A6-BBDE589AF98E,2022-10-14 18:31:34,2022-10-14 18:21:22,2022-10-14 18:31:34,https://www.bseindia.com/xml-data/corpfiling/AttachLive/745d6bd9-e798-419a-8d49-b6beef99ab47.pdf,NaN
2,ea5f274e-405d-4f56-bc1e-7391525a7675.pdf,"Please find attached herewith copy of the transcript of the Analyst / Institutional Investor Meeting held on October 12, 2022. The audio recording of the same is available at https://www.wipro.com/investors/quarterly-results/.",78de1131-13a7-49b7-a722-892582ba3c94,WIPRO LTD. - 507685 - Announcement under Regulation 30 (LODR)-Earnings Call Transcript,wipro,157,A,Company Update,0,2022-10-15T12:05:12.543,...,507685,WIPRO LTD.,00:00:00,13,ANN_507685_78DE1131-13A7-49B7-A722-892582BA3C94,2022-10-15 12:05:12,2022-10-15 18:16:33,2022-10-15 12:05:12,https://www.bseindia.com/xml-data/corpfiling/AttachLive/ea5f274e-405d-4f56-bc1e-7391525a7675.pdf,NaN
3,f9a43851-3d9c-41eb-a9c5-e4f74020536d.pdf,"Enclosed please find the transcript of the Earning Conference Call held on October 12, 2022",fd41bdcd-7a44-4007-945a-1a3c1d034bb8,HCL TECHNOLOGIES LTD. - 532281 - Announcement under Regulation 30 (LODR)-Earnings Call Transcript,hcltech,157,A,Company Update,0,2022-10-17T12:50:20.39,...,532281,HCL TECHNOLOGIES LTD.,00:00:00,17,ANN_532281_FD41BDCD-7A44-4007-945A-1A3C1D034BB8,2022-10-17 12:50:20,2022-10-17 18:08:02,2022-10-17 12:50:20,https://www.bseindia.com/xml-data/corpfiling/AttachLive/f9a43851-3d9c-41eb-a9c5-e4f74020536d.pdf,NaN
4,3baf1c8d-6609-4285-ab99-6dde8b307876.pdf,"Submission of Transcript of Earnings conference call for the second quarter and half year ended September 30, 2022",1e45f9d2-993f-4a1c-8f9b-22a1ad8da1f2,MINDTREE LTD. - 532819 - Announcement under Regulation 30 (LODR)-Earnings Call Transcript,mindtree,157,A,Company Update,0,2022-10-17T18:00:36.283,...,532819,MINDTREE LTD.,00:00:00,17,ANN_532819_1E45F9D2-993F-4A1C-8F9B-22A1AD8DA1F2,2022-10-17 18:00:36,2022-10-17 18:08:01,2022-10-17 18:00:36,https://www.bseindia.com/xml-data/corpfiling/AttachLive/3baf1c8d-6609-4285-ab99-6dde8b307876.pdf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,c037df4d-c831-49d4-a905-18ab176a1921.pdf,"This is to inform you that in terms of the SEBI (Listing Obligations and Disclosure Requirements) Regulations, 2015, the transcript of the Conference Call held on April 20, 2023, post the announcement of the financial results of the Company for the quarter and year ended March 31, 2023, has been uploaded on our Company's website https://www.hcltech.com/investors/results-reports._x000D_\n_x000D_\n A copy of the transcript is

## Getting the audio links
Upon further observations, we notice that if the word <span style="color:red">audio</span> is present in the <span style="color:yellow">HEADLINE</span> column, then either the transcript itself carries the audio link or the audio link is specified in the <span style="color:yellow">MORE</span> column. Hence we just have to notice this and get the audio links and put it in the <span style="color:yellow">AUDIO</span> column of the dataframe. We then save the dataframe into a excel file named final.xlxs

In [5]:
url_pattern = re.compile(r'https?://\S+')
for index, row in audio_df.iterrows():
    if '....' not in row['HEADLINE']:
        urls = re.findall(url_pattern, row['HEADLINE'])
    else:
        urls = re.findall(url_pattern, row['MORE'])
    if len(urls) == 0:
        audio_df.loc[index, 'AUDIO'] = audio_df.loc[index, 'TRANSCRIPTS']
    else:
        audio_df.loc[index, 'AUDIO'] = urls[-1]

transcript_df.loc[audio_df.index, 'AUDIO'] = audio_df['AUDIO']
transcript_df.to_excel('final.xlsx', index=False)
transcript_df

,ATTACHMENTNAME,HEADLINE,NEWSID,NEWSSUB,_company,AGENDA_ID,ANNOUNCEMENT_TYPE,CATEGORYNAME,CRITICALNEWS,DT_TM,...,SCRIP_CD,SLONGNAME,TimeDiff,TotalPageCnt,XML_NAME,disseminated_time,last_update,received_time,TRANSCRIPTS,AUDIO
0,45a5514f-9e10-4004-b787-a093bfac4307.pdf,Transcript of Conference Call held on 12.10.2022,23936ba4-1ab4-4b38-943a-936fe8e2b8db,DELTA CORP LTD. - 532848 - Announcement under Regulation 30 (LODR)-Earnings Call Transcript,deltacorp,157,A,Company Update,0,2022-10-14T16:55:19.01,...,532848,DELTA CORP LTD.,00:00:01,24,ANN_532848_23936BA4-1AB4-4B38-943A-936FE8E2B8DB,2022-10-14 16:55:19,2022-10-14 18:21:23,2022-10-14 16:55:18,https://www.bseindia.com/xml-data/corpfiling/AttachLive/45a5514f-9e10-4004-b787-a093bfac4307.pdf,NaN
1,745d6bd9-e798-419a-8d49-b6beef99ab47.pdf,"Pursuant to Regulation 30 of the SEBI (Listing Obligations and Disclosure Requirements) Regulations, 2015, please find enclosed the transcript of the earnings conference call for the quarter ended September 30, 2022 conducted after the meeting of Board of Directors held on October 10, 2022, for your information and records.",fec1e41a-89a8-4220-91a6-bbde589af98e,TATA CONSULTANCY SERVICES LTD. - 532540 - Announcement under Regulation 30 (LODR)-Earnings Call Transcript,tcs,157,A,Company Update,0,2022-10-14T18:31:34.29,...,532540,TATA CONSULTANCY SERVICES LTD.,00:00:00,24,ANN_532540_FEC1E41A-89A8-4220-91A6-BBDE589AF98E,2022-10-14 18:31:34,2022-10-14 18:21:22,2022-10-14 18:31:34,https://www.bseindia.com/xml-data/corpfiling/AttachLive/745d6bd9-e798-419a-8d49-b6beef99ab47.pdf,NaN
2,ea5f274e-405d-4f56-bc1e-7391525a7675.pdf,"Please find attached herewith copy of the transcript of the Analyst / Institutional Investor Meeting held on October 12, 2022. The audio recording of the same is available at https://www.wipro.com/investors/quarterly-results/.",78de1131-13a7-49b7-a722-892582ba3c94,WIPRO LTD. - 507685 - Announcement under Regulation 30 (LODR)-Earnings Call Transcript,wipro,157,A,Company Update,0,2022-10-15T12:05:12.543,...,507685,WIPRO LTD.,00:00:00,13,ANN_507685_78DE1131-13A7-49B7-A722-892582BA3C94,2022-10-15 12:05:12,2022-10-15 18:16:33,2022-10-15 12:05:12,https://www.bseindia.com/xml-data/corpfiling/AttachLive/ea5f274e-405d-4f56-bc1e-7391525a7675.pdf,https://www.wipro.com/investors/quarterly-results/.
3,f9a43851-3d9c-41eb-a9c5-e4f74020536d.pdf,"Enclosed please find the transcript of the Earning Conference Call held on October 12, 2022",fd41bdcd-7a44-4007-945a-1a3c1d034bb8,HCL TECHNOLOGIES LTD. - 532281 - Announcement under Regulation 30 (LODR)-Earnings Call Transcript,hcltech,157,A,Company Update,0,2022-10-17T12:50:20.39,...,532281,HCL TECHNOLOGIES LTD.,00:00:00,17,ANN_532281_FD41BDCD-7A44-4007-945A-1A3C1D034BB8,2022-10-17 12:50:20,2022-10-17 18:08:02,2022-10-17 12:50:20,https://www.bseindia.com/xml-data/corpfiling/AttachLive/f9a43851-3d9c-41eb-a9c5-e4f74020536d.pdf,NaN
4,3baf1c8d-6609-4285-ab99-6dde8b307876.pdf,"Submission of Transcript of Earnings conference call for the second quarter and half year ended September 30, 2022",1e45f9d2-993f-4a1c-8f9b-22a1ad8da1f2,MINDTREE LTD. - 532819 - Announcement under Regulation 30 (LODR)-Earnings Call Transcript,mindtree,157,A,Company Update,0,2022-10-17T18:00:36.283,...,532819,MINDTREE LTD.,00:00:00,17,ANN_532819_1E45F9D2-993F-4A1C-8F9B-22A1AD8DA1F2,2022-10-17 18:00:36,2022-10-17 18:08:01,2022-10-17 18:00:36,https://www.bseindia.com/xml-data/corpfiling/AttachLive/3baf1c8d-6609-4285-ab99-6dde8b307876.pdf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,c037df4d-c831-49d4-a905-18ab176a1921.pdf,"This is to inform you that in terms of the SEBI (Listing Obligations and Disclosure Requirements) Regulations, 2015, the transcript of the Conference Call held on April 20, 2023, post the announcement of the financial results of the Company for the quarter and year ended March 31, 2023, has been uploaded on our Company's website https://www.hcltech.com/investors/results-report